# 使用扫描作业优化超参数

有许多机器学习算法需要超参数（影响训练但无法通过训练数据本身确定的参数值）。 例如，训练逻辑回归模型时，可以使用正则化率超参数来抵消模型中的偏差；或者训练卷积神经网络时，可以使用学习速率和批大小等超参数来分别控制权重的调整方式以及以小批量方式处理的数据项数量  。 超参数值的选择会显著影响已训练模型的性能或训练该模型所需的时间；通常需要尝试多种组合以找到最佳解决方案。 

## 准备工作

需要最新版本的 azureml-ai-ml 包才能运行此笔记本中的代码。 运行下面的单元以验证是否已安装它。

> **注意**：
> 如果未安装 azure-ai-ml 包，请运行 `pip install azure-ai-ml` 以进行安装。

In [ ]:
## 连接到工作区

安装必需的 SDK 包后，就可以连接到工作区了。

若要连接到工作区，我们需要标识符参数 - 订阅 ID、资源组名称和工作区名称。 已为你填写资源组名称和工作区名称。 只需订阅 ID 即可完成命令。

若要查找所需的参数，请单击工作室右上角的订阅和工作区名称。 右侧将打开一个窗格。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 复制订阅 ID，并将“YOUR-SUBSCRIPTION-ID”替换为复制的值。 </p>

## 创建训练脚本
想要训练机器学习模型但又想改变输入参数时，进行超参数调整是理想的选择。 你需要创建一个训练脚本，该脚本需要一个表示算法超参数之一的输入参数。

运行以下单元以创建 src 文件夹和训练脚本。

请注意，训练脚本需要两个输入参数：

- `--training_data` 应该是一个字符串。 你将指定注册数据资产的路径作为输入训练数据。
- `--reg_rate` 应该是一个数字，但默认值为 `0.01`。 你将使用此输入参数进行超参数调整。

In [ ]:
## 配置并运行命令作业

运行以下单元以训练分类模型来预测糖尿病。 通过运行 src 文件夹中的 train\.py 脚本训练该模型 。 它使用注册的 `diabetes-data` 数据资产作为训练数据。 

- `code`：指定包含要运行的脚本的文件夹。
- `command`：指定要确切运行的内容。
- `environment`：指定在运行命令之前要在计算上安装的必要包。
- `compute`：指定用于运行命令的计算。
- `display_name`：单个作业的名称。
- `experiment_name`：作业所属的试验名称。

请注意，命令作业仅运行一次训练脚本，正则化率为 `0.1`。 在运行扫描作业以调整超参数之前，最佳做法是测试脚本与命令作业是否按预期运行。

In [ ]:
## 定义搜索空间

命令作业成功完成后，可以配置并运行扫描作业。 

首先，需要为超参数指定搜索空间。 要训练三个模型，每个模型具有不同的正则化率（`0.01`、`0.1` 或 `1`），可以使用 `Choice` 超参数定义搜索空间。 

## 配置并提交扫描作业

你将使用扫描函数对训练脚本进行超参数调整。 要配置扫描作业，需要配置以下内容：

- `compute`：要在其上执行作业的计算目标的名称。
- `sampling_algorithm`：要在搜索空间上使用的超参数采样算法。 允许的值为 `random`、`grid` 和 `bayesian`。
- `primary_metric`：每个试运行作业报告的主要指标的名称。 必须使用 `mlflow.log_metric()` 以相同的对应指标名称在用户的训练脚本中记录该指标。
- `goal`：`primary_metric` 的优化目标。 允许值为 `maximize` 和 `minimize`。
- `limits`：扫描作业的限制。 例如，要训练的最大试验次数或模型次数。

请注意，命令作业用作扫描作业的基础。 扫描作业会重用命令作业的配置。

In [ ]:
运行以下单元以提交扫描作业。

In [ ]:
作业完成后，导航至作业概述。 “试用”选项卡将显示所有已训练的模型，以及尝试的每个正则化率值的 `Accuracy` 分数差异。

## Configure and run a command job

Run the cell below to train a classification model to predict diabetes. The model is trained by running the **train\.py** script that can be found in the **src** folder. It uses the registered `diabetes-data` data asset as the training data. 

- `code`: specifies the folder that includes the script to run.
- `command`: specifies what to run exactly.
- `environment`: specifies the necessary packages to be installed on the compute before running the command.
- `compute`: specifies the compute to use to run the command.
- `display_name`: the name of the individual job.
- `experiment_name`: the name of the experiment the job belongs to.

Note that the command job only runs the training script once, with a regularization rate of `0.1`. Before you run a sweep job to tune hyperparameters, it's a best practice to test whether your script works as expected with a command job.

In [ ]:
from azure.ai.ml import command, Input
from azure.ai.ml.constants import AssetTypes

# configure job

job = command(
    code="./src",
    command="python train.py --training_data ${{inputs.diabetes_data}} --reg_rate ${{inputs.reg_rate}}",
    inputs={
        "diabetes_data": Input(
            type=AssetTypes.URI_FILE, 
            path="azureml:diabetes-data:1"
            ),
        "reg_rate": 0.01,
    },
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-mlflow",
    experiment_name="diabetes-training", 
    tags={"model_type": "LogisticRegression"}
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

## Define the search space

When your command job has completed successfully, you can configure and run a sweep job. 

First, you'll need to specify the search space for your hyperparameter. To train three models, each with a different regularization rate (`0.01`, `0.1`, or `1`), you can define the search space with a `Choice` hyperparameter. 

In [ ]:
from azure.ai.ml.sweep import Choice

command_job_for_sweep = job(
    reg_rate=Choice(values=[0.01, 0.1, 1]),
)

## Configure and submit the sweep job

You'll use the sweep function to do hyperparameter tuning on your training script. To configure a sweep job, you'll need to configure the following:

- `compute`: Name of the compute target to execute the job on.
- `sampling_algorithm`: The hyperparameter sampling algorithm to use over the search space. Allowed values are `random`, `grid` and `bayesian`.
- `primary_metric`: The name of the primary metric reported by each trial job. The metric must be logged in the user's training script using `mlflow.log_metric()` with the same corresponding metric name.
- `goal`: The optimization goal of the `primary_metric`. The allowed values are `maximize` and `minimize`.
- `limits`: Limits for the sweep job. For example, the maximum amount of trials or models you want to train.

Note that the command job is used as the base for the sweep job. The configuration for the command job will be reused by the sweep job.

In [ ]:
# apply the sweep parameter to obtain the sweep_job
sweep_job = command_job_for_sweep.sweep(
    compute="aml-cluster",
    sampling_algorithm="grid",
    primary_metric="training_accuracy_score",
    goal="Maximize",
)

# set the name of the sweep job experiment
sweep_job.experiment_name="sweep-diabetes"

# define the limits for this sweep
sweep_job.set_limits(max_total_trials=4, max_concurrent_trials=2, timeout=7200)

Run the following cell to submit the sweep job.

In [ ]:
returned_sweep_job = ml_client.create_or_update(sweep_job)
aml_url = returned_sweep_job.studio_url
print("Monitor your job at", aml_url)

When the job is completed, navigate to the job overview. The **Trials** tab will show all models that have been trained and how the `Accuracy` score differs for each regularization rate value you tried.